# Homework assignment 1: Markov Chain

## Problem 1
Read the article in https://www.geeksforgeeks.org/markov-chains-in-nlp/, and answer the following questions.

* (a) What are N-grams of an input sequence?  
        A contiguous sequence of n items (words or characters) from a given sample of text.
* (b) How to determine the probability of each element in the transition matrix?  
        Each element (i,j) of the matrix represents the probability of moving from state i to state j.
        For example, if you’re in state 1 and want to move to state 2 or state 3, you would look at the row for state 1 and the columns for state 2 and state 3 to find the probabilities.
* (c) If you want to increase the variety of the sequence generation (every time the outputs are different), what kinds of properties the training texts should be?  
        The probabilities of moving from one word to another should be balanced.

## Problem 2
Try the following codes, and answer questions.

In [1]:
# Install the required packages
#!pip install nltk
# !pip install numpy

In [1]:
import re
from nltk import ngrams
import numpy as np
from typing import List, Tuple, Union, Dict
from itertools import product

In [2]:
def words_to_index(words: List[str], base: int, unique_words: Dict[str, int]) -> int:
    """
        Convert a list of words to an index representation.

        Args:
            words (List[str]): The list of words to be converted.
            base (int): The base value used for conversion.
            unique_words (Dict[str, int]): A dictionary mapping unique words to their corresponding indices.

        Returns:
            - int: The index representation of the given list of words.
        
        Example:
            >>> words_to_index(["the", "quick", "brown", "fox"], 10, {"the": 0, "quick": 1, "brown": 2, "fox": 3})
            123
    """
    length = len(words)
    numbers = [unique_words[word] for word in words]
    return sum([num * (base ** (length - 1 - idx)) for idx, num in enumerate(numbers)])

def index_to_words(index: int, base: int, length: int, unique_words: Dict[str, int]):
    """
        Converts an index to a list of words based on a given base and unique words.

        Args:
            index (int): The index to convert.
            base (int): The base used for conversion.
            length (int): The length of the resulting list.
            unique_words (Dict[str, int]): A dictionary mapping unique words to their corresponding indices.

        Returns:
            List[str]: A list of words corresponding to the given index.

        Examples:
            >>> unique_words = {'apple': 0, 'banana': 1, 'cherry': 2}
            >>> index_to_words(5, 3, 2, unique_words)
            ['banana', 'cherry'] (Because 5 = 1 * 3^1 + 2 * 2^0)

            >>> unique_words = {'red': 0, 'green': 1, 'blue': 2}
            >>> index_to_words(2, 3, 1, unique_words)
            ['blue'] (Because 2 = 2 * 3^0)
    """
    numbers = []
    
    unique_words_list = list(unique_words.keys())
    for pow in range(length - 1, -1, -1):
        numbers.append(index // (base ** pow))
        index -= numbers[-1] * (base ** pow)
    
    return [unique_words_list[num] for num in numbers]

### Step 1: Remove some unnecessary characters

In [3]:
def remove_unnecessary_characters(text: str):
    """
        Removes unnecessary characters from the given text and converts it to lowercase.

        Args:
            text (str): The input text to be processed.
        Returns:
            str: The processed text with unnecessary characters removed and converted to lowercase.
    """
    # Remove unnecessary characters
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # Convert to lowercase
    text = text.lower()

    return text

### Step 2: Generate N-grams

In [4]:
def generate_n_grams(text: str, n: int):
    """
        Generate n-grams from the given text.

        Args:
            text (str): The input text from which n-grams will be generated.
            n (int): The number of consecutive words in each n-gram.
        Returns:
            List[Tuple[str]]: A list of tuples representing the generated n-grams.
    """
    # [TODO] Generate n-grams, using the ngrams function from the nltk library
    n_grams = ngrams(text.split(),n)
    print("n grams",n_grams)
    # Convert to list of tuples
    n_grams = list(n_grams)

    return n_grams

### Step 3: Compute Transition Matrix

In [5]:
def compute_transition_matrix(n_grams: List[Tuple[str]]):
    """
        Computes the transition matrix and unique states for a given list of n-grams.
            
        Args:
            n_grams (List[Tuple[str]]): A list of n-grams, where each n-gram is a tuple of strings.
        Returns:
            np.ndarray: transition matrix.
            Dict[str, int]: A dictionary mapping unique words to their corresponding indices.
    """
    
    # Get the value of n
    n = len(n_grams[0])
    
    ## Step 3.1: Collect all possible words (label words with indices)
    
    unique_words = {}
    unique_words_count = 0
    
    # A helper function to add a word to the unique_words dictionary, if it is not already present
    def _add_to_unique_word(word: str):
        # We need to increase the counter outside this function
        nonlocal unique_words_count

        # Check if the state is already in the unique_states dictionary
        if unique_words.get(word, None) is None:
            unique_words[word] = unique_words_count
            unique_words_count += 1
    
    # Iterate over all n-grams to collect all possible words
    for n_gram in n_grams:
        for word in n_gram:
            # [TODO] Add the word to the unique_words dictionary
            # hint: use the _add_to_unique_word function
            _add_to_unique_word(word)
    
    
    ## Step 3.2: Compute the transition matrix
    
    # In this part, we use `words_to_index` and `index_to_words` functions to convert between state name (words) and indices
    
    # Example:
    # If we have 3 unique words: 0 for "apple", 1 for "banana", 2 for "cherry", and n = 3,
    # then the state names are "apple, apple", "apple, banana", "apple, cherry", "banana, apple", "banana, banana", and so on.
    # In this case, the mapping from states to indices would be:
    # "(apple, apple)" <-> 0, "(apple, banana)" <-> 1, "(apple, cherry)" <-> 2, "(banana, apple)" <-> 3, and so on.
    
    # [TODO] Compute the number of states in Markov chain. Number of states is the number of unique words to the power of n-1
    unique_states_count = unique_words_count ** (n-1)
    
    # [TODO] Create transition matrix, filled with zeros (the size of the transition matrix is # states x # states)
    transition_matrix = np.zeros((unique_states_count, unique_states_count))

    # Count the number of transitions from each state to another state
    for n_gram in n_grams:
        # [TODO] Find the indices of the states (from and to) (i.e. the first n-1 words and the last n-1 words)
        # hint: use words_to_index function
        state_from_index = words_to_index(n_gram[:-1], unique_words_count, unique_words)
        state_to_index = words_to_index(n_gram[1:],unique_words_count , unique_words)

        # Increment the count of the transition from state_from to state_to
        transition_matrix[state_from_index][state_to_index] += 1

    # Special case:
    # if the sum of number of transition of a state is zero, which means it has no information about the next state,
    # we set the probabilities of transitioning to all possible next states to 1 / # words
    
    # Check if any row sums to zero
    row_sums_is_zero = transition_matrix.sum(axis=1) == 0
    
    
    # Iterate over rows to find zero-sum rows
    first_word = list(unique_words.keys())[0]
    
    for idx, is_zero in enumerate(row_sums_is_zero):
        # If the row sum is zero
        if is_zero:
            # Convert the index to state representation
            state_name = index_to_words(idx, unique_words_count, n-1, unique_words)
            
            # Calculate the range of indices for the next posiible states
            possible_states_start_index = words_to_index(state_name[1:] + [first_word, ], unique_words_count, unique_words)
            possible_states_end_index = possible_states_start_index + unique_words_count
            
            # Set the number of transitions to 1 for the range
            transition_matrix[idx, possible_states_start_index : possible_states_end_index] = 1

    ## Step 3.3: Normalize the transition matrix
    
    # The following steps is to normalize transition matrix so that the sum of each row is 1
    # (The reason is: By the definition of a Markov chain, the sum of each row should be 1)
    
    # [TODO] Compute the sum of each row
    # hint: use np.sum function and check if the axis parameter is set correctly
    row_sums =transition_matrix.sum(axis = 1)
    
    # [TODO] Divide the transition matrix by the sum of each row to make the sum of each row equals to 1
    transition_matrix =  transition_matrix / row_sums[:,np.newaxis]

    return transition_matrix, unique_words

### Step 4: Generate text

In [7]:
def generate(unique_words: Dict[str, int], transition_matrix: np.ndarray, start_from: Union[str, List[str]], n: int, length: int=10):
    """
        Generate text using a Markov chain model.

        Args:
            unique_states (Dict[str, int]): A dictionary mapping unique words to their corresponding indices.
            transition_matrix (np.ndarray): A numpy array representing the transition probabilities between states.
            start_from (Union[str, List[str]]): The starting state(s) for text generation.
            n (int): The size of the grams.
            length (int, optional): The length of the generated text. Defaults to 10.

        Returns:
            The generated text.
    """
    # Generate text
    generated_words = start_from.copy() if type(start_from) is list else [start_from]
    
        
    # Assert if the number of start words does not equal to n-1
    assert len(generated_words) >= n-1, "The number of start words should be greater than or equals to n-1 ({})".format(n-1)

    # Get the number of unique words    
    unique_words_count = len(unique_words)
    
    # [TODO] Get the number of unique states
    # hint: check step 3.2 in the compute_transition_matrix function
    unique_states_count = unique_words_count ** (n-1)
    # Generate the next words
    for _ in range(length):
        # [TODO] Get index of current states
        # hint: The current states (current words) is the last n-1 words in the generated text
        # hint: use words_to_index function
        current_words_index = words_to_index(generated_words[-(n-1):],unique_words_count, unique_words)

        # [TODO] Get probability distribution for next state, using the information in the transition matrix
        probabilities = transition_matrix[current_words_index]

        # Select next word based on probabilities, using np.random.choice function
        next_words_index = np.random.choice(unique_states_count, p=probabilities)

        # [TODO] Decode the index and get the last word
        # hint: use index_to_words function
        next_word = index_to_words(next_words_index,unique_words_count,n-1,unique_words)[-1]

        # Add next word to generated text
        generated_words.append(next_word)

    # return generated string
    return ' '.join(generated_words)

In [8]:
# [TODO] Change the text below and try different values of n
# text = "i love cat. cat love me. But I love dog too and dog also love me"
text = """the cat sleeps on the warm mat. birds sing in tall trees. children play in the park nearby. 
cars rush down busy streets. a cool breeze blows through open windows."""

n = 3

# Process the text and generate the transition matrix
text = remove_unnecessary_characters(text)
n_grams = generate_n_grams(text, n)
transition_matrix, unique_words = compute_transition_matrix(n_grams)

n grams <generator object ngrams at 0x000002065D601740>


In [9]:
# Print the transition matrix and unique states for obsevation
print("===== The indices for unique states are: =====")
unique_words_count = len(unique_words)
for word_name in list(product(*[unique_words for _ in range(n-1)]))[:20]:
    print(f"{','.join(word_name):10s}: {words_to_index(word_name, unique_words_count, unique_words)}")
print("...", end="\n\n")

print("===== The transition matrix is (Shape of trasition matrix: {}): =====".format(transition_matrix.shape))
print(transition_matrix)
print()

===== The indices for unique states are: =====
the,the   : 0
the,cat   : 1
the,sleeps: 2
the,on    : 3
the,warm  : 4
the,mat   : 5
the,birds : 6
the,sing  : 7
the,in    : 8
the,tall  : 9
the,trees : 10
the,children: 11
the,play  : 12
the,park  : 13
the,nearby: 14
the,cars  : 15
the,rush  : 16
the,down  : 17
the,busy  : 18
the,streets: 19
...

===== The transition matrix is (Shape of trasition matrix: (729, 729)): =====
[[0.03703704 0.03703704 0.03703704 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.03703704 0.03703704 0.03703704]]



In [7]:
# [TODO] Write down 3 or more initial words and length of generated text to start the text generation
# experiments = [
#     # Each entry is (Initial words, the length of the generated text)
#     ('i love', 10), ('cat love', 15), ('love me', 30)
    
# ]

experiments = [
    ('the cat', 10),
    ('birds sing', 15),
    ('children play in', 20)
]


for idx, (start_from, length) in enumerate(experiments, 1):
    start_from = start_from.split(" ")

    # Generate text using the transition matrix
    generated_text = generate(unique_words, transition_matrix, start_from, n, length=length)
    
    # Print out the generated text
    print("#{} (length={}): {}".format(idx, length, generated_text))

#1 (length=10): the cat sleeps on the warm mat birds sing in tall trees
#2 (length=15): birds sing in tall trees children play in the park nearby cars rush down busy streets a
#3 (length=20): children play in the park nearby cars rush down busy streets a cool breeze blows through open windows streets park open cat streets


### Answer the following questions

* (a) Write a new text of at least 15 words as the input.  
    the cat sleeps on the warm mat. birds sing in tall trees. children play in the park nearby. cars rush down busy streets.        
    a cool breeze blows through open windows.
* (b) Run the program 3 times with different output length and different initial words. Show the outputs.
    * (Please modify the `experiments` at the cell above and run the cell)  
    
    #1 (length=10): the cat sleeps on the warm mat birds sing in tall trees  
    #2 (length=15): birds sing in tall trees children play in the park nearby cars rush down busy streets a  
    #3 (length=20): children play in the park nearby cars rush down busy streets a cool breeze blows through open windows streets park open cat streets  
    
* (c) Try different N of N-grams. How the N influences the output sequence?  
    The value of N in N-grams significantly influences the output sequence. As N increases, the generated text becomes more fluent and coherent, closely resembling natural human language. Larger N-grams capture more extended word sequences, preserving the original text's structure and context. Conversely, smaller N values produce more random and often incomprehensible text due to limited context. However, while larger N values enhance fluency, they may limit creativity by reproducing longer sequences from the original text. Smaller N values, despite reduced coherence, allow for more unexpected word combinations. The optimal N value often balances fluency and creativity, depending on the specific requirements of the text generation task.


## Problem 3
The Stationary Distribution of a Markov chain is a distribution of probabilities that remains unchanged after a transition from one state to another.

### (a)  Ask an LLM (Large Language Model), such as ChatGPT, what are the applications of stationary distribution of a Markov chain. You need to show which prompts are used, and state how you verify the correctness of the results (output by LLMs).

#### Prompt used:
What are the main applications of the stationary distribution of a Markov chain? Please provide several examples from different fields.

#### LLM's response:

1. Web Page Ranking (PageRank Algorithm):
   - Application: Search engine result ranking
   - Role: Web pages are viewed as states in a Markov chain, with links as transition probabilities. The stationary distribution represents the importance or weight of each web page.

2. Genetic Sequence Analysis:
   - Application: Predicting gene structure in DNA sequences
   - Role: Using Hidden Markov Models, the stationary distribution helps identify coding and non-coding regions.

3. Financial Market Modeling:
   - Application: Stock price prediction, risk assessment
   - Role: Market states are viewed as a Markov chain, with the stationary distribution reflecting long-term market trends.

#### Verification methods:
1. Cross-validation: Consult reliable academic resources (textbooks, journal articles)
2. Logical consistency check: Ensure logical soundness and consistency with Markov chain principles
3. Case studies: Look for actual research or industrial application cases

### (b) Ask an LLM, such as ChatGPT, what numerical method is the most efficient approach to compute the stationary distribution? You need to show which prompts are used, and state how you verify the correctness of the results (output by LLMs)

#### Prompt used:
What numerical method is the most efficient approach to compute the stationary distribution?

#### LLM's response:

To determine the most efficient numerical method for computing the stationary distribution, we need to consider a few factors:

- The size and sparsity of the transition matrix
- The desired accuracy
- The available computational resources

Generally, for Markov chains with reasonably sized transition matrices, the power method is often considered one of the most efficient approaches. Here's a brief overview:

Power Method:
- Simple to implement
- Works well for sparse matrices
- Converges relatively quickly for many practical problems

However, for very large or ill-conditioned matrices, other methods may be more suitable:

Iterative methods:
- Gauss-Seidel method
- Jacobi method
These can be faster for certain types of matrices

Linear algebra approaches:
- Eigenvalue decomposition
- Solving linear systems
Can be very fast for smaller matrices, but may become computationally expensive for large ones

The "most efficient" method can vary depending on the specific problem. For many practical applications, the power method offers a good balance of simplicity and efficiency. However, if you're dealing with very large or specialized problems, you might need to experiment with different methods to find the most efficient one for your specific case.

#### Verification methods:
1. Software Package Analysis:
- Investigate popular numerical computation software packages (e.g., MATLAB, NumPy, SciPy) to see which methods they use as default for computing stationary distributions.
- Compare the choices made by these established software packages with the LLM's recommendations.
- Check the documentation of these packages for any performance comparisons or recommendations for different types of matrices.
2. Case Studies:
- Search for real-world applications where stationary distributions are computed.
- Examine which methods are commonly used in practice for different types of Markov chains.
- Compare the methods used in these case studies with those suggested by the LLM.
### (c) Implement the method suggested by the LLM. Use the transition matrix generated in question 2.a as an input to compute its stationary distribution.
[TODO: Implement the power method as suggested by the LLM]
def compute_stationary_distribution(transition_matrix):
    # Implement the power method here
    pass

### (d) The theory of probability matrix is given in the textbook 6.8, Eigenvalues/eigenvector of nonegtive matrices. Read textbook 6.8 and ask two questions that you are curious most about it

1. How does the concept of a regular Markov process contribute to our understanding of the long-term behavior of stochastic systems?

2. In what way does Perron's theorem support the analysis of convergence properties in positive stochastic matrices, particularly in establishing the dominance of the eigenvalue λ = 1?

In [11]:
def compute_stationary_distribution(transition_matrix: np.ndarray) -> np.ndarray:
    """
        Compute the stationary distribution of a Markov chain.

        Parameters:
            transition_matrix (np.ndarray): A numpy array representing the transition probabilities between states.
        Returns:
            The stationary distribution of the Markov chain.
    """
    # [TODO] Implement the function to compute the stationary distribution of a Markov chain, using the method suggested by the LLM
    n = transition_matrix.shape[0]
    stationary_distribution = np.ones(n) / n
    epsilon = 1e-8
    max_iterations = 1000
    for _ in range(max_iterations):
        new_distribution = stationary_distribution @ transition_matrix
        
        if np.allclose(new_distribution,stationary_distribution,epsilon):
            return new_distribution 
        stationary_distribution = new_distribution
    return stationary_distribution

In [12]:
# Compute the stationary distribution for the transition matrix obtained from the previous problem
stationary_distribution = compute_stationary_distribution(transition_matrix)

# Convert the stationary distribution to float16 to prevent something like -1.2345678e-16 but it is actually 0
print(stationary_distribution.astype(np.float16))

[0.0001633 0.0001633 0.0001633 ... 0.0001754 0.0001754 0.0001754]
